<a href="https://colab.research.google.com/github/hsandaver/hsandaver/blob/main/Machine_Learning_Fading_Simulator_Complex_Model_with_Hyperparameters.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# -*- coding: utf-8 -*-
"""
Updated Script with Enhanced Machine Learning Model:
- Replaced Random Forest with XGBoost
- Added Hyperparameter Tuning with GridSearchCV
- Implemented Cross-Validation
- Enhanced Synthetic Data Generation
- Added Feature Engineering with Polynomial Features
"""

import sys
import subprocess
import importlib
import logging

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(message)s')

# Function to install packages
def install_packages(packages):
    for package in packages:
        if package == 'google.colab':
            continue  # Skip installation as it's already available in Colab
        try:
            importlib.import_module(package)
            logging.info(f"Package '{package}' is already installed.")
        except ImportError:
            print(f"Installing package: {package}")
            subprocess.check_call([sys.executable, "-m", "pip", "install", package])

# List of required packages
required_packages = [
    'scikit-image', 'numpy', 'pandas', 'matplotlib',
    'Pillow', 'scipy', 'sklearn', 'ipywidgets', 'xgboost'
]
install_packages(required_packages)

# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
from skimage import color
from skimage.color import deltaE_ciede2000
from sklearn.model_selection import train_test_split, GridSearchCV, KFold, cross_val_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PolynomialFeatures, MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score
import ipywidgets as widgets
from IPython.display import display
from ipywidgets import VBox, Layout
from xgboost import XGBRegressor
from sklearn.multioutput import MultiOutputRegressor

# For file uploads and downloads in Colab
try:
    from google.colab import files
except ImportError:
    # Define dummy functions if not in Colab
    class DummyFiles:
        def upload(self):
            print("File upload functionality is not implemented in this environment.")
            return {}
        def download(self, filename):
            print(f"File download functionality is not available. File '{filename}' saved locally.")
    files = DummyFiles()

# -----------------------------
# Step 1: Data Upload and Preprocessing
# -----------------------------

def upload_file(prompt_message, file_types=None):
    print(prompt_message)
    uploaded = files.upload()
    if not uploaded:
        logging.error("No file uploaded.")
        sys.exit(1)
    filename = next(iter(uploaded))
    if file_types and not filename.lower().endswith(file_types):
        logging.error(f"Uploaded file must be one of the following types: {file_types}")
        sys.exit(1)
    logging.info(f"Uploaded file: {filename}")
    return filename

def load_and_clean_dataset(csv_filename):
    try:
        dataset = pd.read_csv(csv_filename)
        required_columns = {'L', 'A', 'B', 'Color Name'}
        if not required_columns.issubset(dataset.columns):
            missing = required_columns - set(dataset.columns)
            logging.error(f"Dataset is missing required columns: {missing}")
            sys.exit(1)
        dataset = dataset.replace([np.inf, -np.inf], np.nan).dropna(subset=['L', 'A', 'B'])
        logging.info(f"Dataset loaded with {len(dataset)} entries after cleaning.")
        return dataset
    except Exception as e:
        logging.error(f"Failed to load dataset: {e}")
        sys.exit(1)

def upload_and_process_image():
    image_filename = upload_file("Please upload the image file you want to analyze.", file_types=('.png', '.jpg', '.jpeg'))
    try:
        image = Image.open(image_filename).convert('RGB')
        image_array = np.array(image).astype(np.float32) / 255.0
        lab_image = color.rgb2lab(image_array)
        logging.info(f"Image '{image_filename}' loaded and converted to LAB color space.")
        return image, lab_image
    except Exception as e:
        logging.error(f"Failed to process image: {e}")
        sys.exit(1)

# -----------------------------
# Step 2: Machine Learning Model Training
# -----------------------------

def create_synthetic_data(art_types, material_types, dye_types, valid_combinations, num_samples_per_combination=500, random_seed=42):
    np.random.seed(random_seed)
    data_list = []
    for art_type in art_types:
        for material_type in material_types:
            dye_type_options = [dye for art, material, dye in valid_combinations
                                if art == art_type and material == material_type]
            dye_type_options = [dye for dye in dye_type_options if dye is not None]
            if not dye_type_options:
                dye_type_options = [None]
            for dye_type in dye_type_options:
                lux_hours = np.random.uniform(low=1000, high=100000, size=num_samples_per_combination)
                uv_exposure = np.random.uniform(low=0.0, high=1.0, size=num_samples_per_combination)
                temperature = np.random.uniform(low=-10, high=50, size=num_samples_per_combination)
                humidity = np.random.uniform(low=0, high=100, size=num_samples_per_combination)
                exposure_time_years = lux_hours / 8760  # Convert to years assuming 8760 hours/year

                # Generate synthetic fading data based on art type, material type, and dye type
                L_fading, A_fading, B_fading = generate_fading_data(
                    art_type, material_type, dye_type, lux_hours, uv_exposure, temperature, humidity, num_samples_per_combination
                )
                art_type_array = np.array([art_type]*num_samples_per_combination)
                material_type_array = np.array([material_type]*num_samples_per_combination)
                dye_type_array = np.array([dye_type if dye_type else 'None']*num_samples_per_combination)

                data = pd.DataFrame({
                    'art_type': art_type_array,
                    'material_type': material_type_array,
                    'dye_type': dye_type_array,
                    'lux_hours': lux_hours,
                    'uv_exposure': uv_exposure,
                    'temperature': temperature,
                    'humidity': humidity,
                    'L_fading': L_fading,
                    'A_fading': A_fading,
                    'B_fading': B_fading
                })
                data_list.append(data)
    synthetic_data = pd.concat(data_list, ignore_index=True)
    logging.info("Synthetic data generated for valid art, material, and dye type combinations.")
    return synthetic_data

def generate_fading_data(art_type, material_type, dye_type, lux_hours, uv_exposure, temperature, humidity, num_samples):
    # Base fading values
    L_fading = np.zeros(num_samples)
    A_fading = np.zeros(num_samples)
    B_fading = np.zeros(num_samples)

    # Normalize lux_hours and uv_exposure
    lux_normalized = lux_hours / 100000  # Now ranges from 0.01 to 1
    uv_normalized = uv_exposure  # Already ranges from 0.0 to 1.0

    # Combine into a single exposure factor, scaled appropriately
    exposure_factor = lux_normalized + uv_normalized

    # Limit exposure factor to a maximum value to prevent excessive fading
    exposure_factor = np.clip(exposure_factor, 0, 2)  # Max exposure factor is 2

    # Refined Material and Dye Interactions with Light (Lux Hours and UV)
    if art_type == 'Chromolithograph Print':
        base_L_fading = np.random.normal(loc=-3, scale=1.5, size=num_samples)
        base_A_fading = np.random.normal(loc=-2, scale=1, size=num_samples)
        base_B_fading = np.random.normal(loc=-2, scale=1, size=num_samples)
        # Adjust for exposure factor
        L_fading += base_L_fading * exposure_factor
        A_fading += base_A_fading * exposure_factor
        B_fading += base_B_fading * exposure_factor
    elif art_type == 'Sanguine Etching':
        base_A_fading = np.random.normal(loc=-3, scale=1.5, size=num_samples)
        # Adjust for lux hours (normalized)
        A_fading += base_A_fading * lux_normalized
    elif art_type == 'Steel Engraving':
        base_L_fading = np.random.normal(loc=-1, scale=0.5, size=num_samples)
        # Adjust for lux hours (normalized)
        L_fading += base_L_fading * lux_normalized
    elif art_type == 'None':
        # No artwork-specific fading
        pass

    # Adjust fading based on material type
    if 'Acidic' in material_type:
        # Incorporating Material Yellowing and Paper Acidification
        L_fading += np.random.normal(loc=-2, scale=1, size=num_samples) * uv_normalized
        B_fading += np.random.normal(loc=3, scale=1, size=num_samples) * uv_normalized  # Increase yellowing
    elif 'Alkaline' in material_type:
        L_fading += np.random.normal(loc=-1, scale=0.5, size=num_samples) * lux_normalized
    elif material_type == 'Textiles':
        if dye_type == 'Natural':
            # Fading Curves for Specific Materials (non-linear fading)
            fading_multiplier = np.log(lux_hours + 1)
            # Scale fading_multiplier to prevent large values
            fading_multiplier = fading_multiplier / np.log(100000 + 1)  # Normalize to 0-1
            L_fading += np.random.normal(loc=-5, scale=1.5, size=num_samples) * uv_normalized * fading_multiplier
            A_fading += np.random.normal(loc=-2, scale=1, size=num_samples) * uv_normalized * fading_multiplier
            B_fading += np.random.normal(loc=-2, scale=1, size=num_samples) * uv_normalized * fading_multiplier
        elif dye_type == 'Synthetic':
            L_fading += np.random.normal(loc=-3, scale=1, size=num_samples) * uv_normalized
            A_fading += np.random.normal(loc=-1, scale=1, size=num_samples) * uv_normalized
            B_fading += np.random.normal(loc=-1, scale=1, size=num_samples) * uv_normalized
    elif material_type == 'Paper with Black Text':
        # Black ink fading
        L_fading += np.random.normal(loc=0, scale=0.5, size=num_samples) * lux_normalized
        A_fading += np.random.normal(loc=0, scale=0.5, size=num_samples) * lux_normalized
        B_fading += np.random.normal(loc=0, scale=0.5, size=num_samples) * lux_normalized

    # Introduce interaction terms and environmental effects
    exposure_interaction = lux_normalized * uv_normalized
    temperature_effect = (temperature / 50.0) * np.random.normal(0, 1, num_samples)
    humidity_effect = (humidity / 100.0) * np.random.normal(0, 1, num_samples)

    # Adjust fading with new factors
    L_fading += exposure_interaction * np.random.normal(-5, 1, num_samples)
    A_fading += temperature_effect * np.random.normal(-2, 1, num_samples)
    B_fading += humidity_effect * np.random.normal(2, 1, num_samples)

    # Limit fading values to realistic ranges
    L_fading = np.clip(L_fading, -20, 0)  # L* can decrease up to 20 units
    A_fading = np.clip(A_fading, -10, 10)  # a* changes within ±10 units
    B_fading = np.clip(B_fading, -10, 10)  # b* changes within ±10 units

    return L_fading, A_fading, B_fading

def prepare_features(synthetic_data):
    X_numeric = synthetic_data[['lux_hours', 'uv_exposure', 'temperature', 'humidity']]
    X_categorical = synthetic_data[['art_type', 'material_type', 'dye_type']]

    # Replace None with string 'None' for encoding
    X_categorical = X_categorical.fillna('None')

    # One-hot encode the categorical variables
    encoder = OneHotEncoder(sparse_output=False)
    X_categorical_encoded = encoder.fit_transform(X_categorical)

    # Create polynomial features
    poly = PolynomialFeatures(degree=2, interaction_only=True, include_bias=False)
    X_numeric_poly = poly.fit_transform(X_numeric)

    # Combine numeric and encoded categorical features
    X = np.hstack((X_numeric_poly, X_categorical_encoded))
    Y = synthetic_data[['L_fading', 'A_fading', 'B_fading']].values
    return X, Y, encoder, poly

def train_ml_model(X, Y):
    scaler = MinMaxScaler()
    X_scaled = scaler.fit_transform(X)

    xgb = XGBRegressor(objective='reg:squarederror', random_state=42)
    multi_xgb = MultiOutputRegressor(xgb)

    param_grid = {
        'estimator__n_estimators': [100, 200],
        'estimator__max_depth': [3, 5, 7],
        'estimator__learning_rate': [0.01, 0.1, 0.2]
    }

    grid_search = GridSearchCV(
        multi_xgb, param_grid, cv=3, scoring='neg_mean_squared_error', verbose=1
    )

    grid_search.fit(X_scaled, Y)
    best_model = grid_search.best_estimator_
    logging.info(f"Best parameters: {grid_search.best_params_}")

    # Cross-validation
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    mse_scores = cross_val_score(
        best_model, X_scaled, Y, cv=kf, scoring='neg_mean_squared_error'
    )
    avg_mse = -np.mean(mse_scores)
    logging.info(f"Cross-validated MSE: {avg_mse:.4f}")

    best_model.fit(X_scaled, Y)
    return best_model, scaler, avg_mse

# -----------------------------
# Step 3: Art Type, Material Type, and Dye Type Selection and Exposure Simulation Functions
# -----------------------------

# Art Types
art_types = [
    'Chromolithograph Print',
    'Sanguine Etching',
    'Steel Engraving',
    'None',  # Added for materials without artwork
]

# Material Types
material_types = [
    'Acidic Wove Paper',
    'Acidic Rag Paper',
    'Alkaline Wove Paper',
    'Alkaline Rag Paper',
    'Textiles',
    'Paper with Black Text',
]

# Dye Types (for Textiles)
dye_types = [
    'Natural',
    'Synthetic',
]

# Valid combinations of Art Type, Material Type, and Dye Type
valid_combinations = [
    # Art Type, Material Type, Dye Type
    ('Chromolithograph Print', 'Acidic Wove Paper', None),
    ('Sanguine Etching', 'Acidic Wove Paper', None),
    ('Sanguine Etching', 'Acidic Rag Paper', None),
    ('Sanguine Etching', 'Alkaline Wove Paper', None),
    ('Sanguine Etching', 'Alkaline Rag Paper', None),
    ('Steel Engraving', 'Acidic Wove Paper', None),
    ('None', 'Textiles', 'Natural'),
    ('None', 'Textiles', 'Synthetic'),
    ('None', 'Paper with Black Text', None),
    ('None', 'Acidic Wove Paper', None),
    ('None', 'Acidic Rag Paper', None),
    ('None', 'Alkaline Wove Paper', None),
    ('None', 'Alkaline Rag Paper', None),
    # Add other valid combinations as necessary
]

# Create Art Type dropdown
art_type_dropdown = widgets.Dropdown(
    options=art_types,
    value=art_types[0],
    description='Art Type:',
    disabled=False,
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='500px')
)

# Create Material Type dropdown
material_type_dropdown = widgets.Dropdown(
    options=[],
    description='Material Type:',
    disabled=False,
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='500px')
)

# Create Dye Type dropdown (initially hidden)
dye_type_dropdown = widgets.Dropdown(
    options=[],
    value=None,
    description='Dye Type:',
    disabled=False,
    layout=widgets.Layout(visibility='hidden', width='500px'),
    style={'description_width': 'initial'}
)

# Function to update Material Type options based on selected Art Type
def update_material_type_options(*args):
    selected_art_type = art_type_dropdown.value
    valid_materials = [material for art, material, dye in valid_combinations if art == selected_art_type]
    valid_materials = sorted(set(valid_materials))
    material_type_dropdown.options = valid_materials
    if material_type_dropdown.value not in valid_materials:
        material_type_dropdown.value = valid_materials[0] if valid_materials else None
    update_dye_type_visibility()

def update_dye_type_visibility(*args):
    selected_material_type = material_type_dropdown.value
    if selected_material_type == 'Textiles':
        dye_type_dropdown.layout.visibility = 'visible'
        valid_dyes = [dye for art, material, dye in valid_combinations
                      if material == selected_material_type and dye is not None]
        valid_dyes = sorted(set(valid_dyes))
        dye_type_dropdown.options = valid_dyes
        if dye_type_dropdown.value not in valid_dyes:
            dye_type_dropdown.value = valid_dyes[0] if valid_dyes else None
    else:
        dye_type_dropdown.layout.visibility = 'hidden'
        dye_type_dropdown.options = []
        dye_type_dropdown.value = None

# Attach the update functions to the dropdowns
art_type_dropdown.observe(update_material_type_options, names='value')
material_type_dropdown.observe(update_dye_type_visibility, names='value')

# Initial update of Material Type options
update_material_type_options()

# Arrange dropdowns vertically with spacing
dropdown_layout = Layout(display='flex', flex_flow='column', align_items='stretch', width='500px')
dropdowns = VBox([art_type_dropdown, material_type_dropdown, dye_type_dropdown], layout=dropdown_layout)

# Display the dropdowns
display(dropdowns)

# Create the sliders with adjusted style and layout
time_slider = widgets.FloatSlider(
    value=5,
    min=0,
    max=100,
    step=1,
    description='Years of Aging:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='500px')
)
uv_slider = widgets.FloatSlider(
    value=0.5,
    min=0.0,
    max=1.0,
    step=0.01,
    description='UV Exposure:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='500px')
)
lux_slider = widgets.FloatSlider(
    value=50000,
    min=0,
    max=100000,
    step=1000,
    description='Lux Hours:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='500px')
)
humidity_slider = widgets.FloatSlider(
    value=50,
    min=0,
    max=100,
    step=1,
    description='Humidity (%):',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='500px')
)
temp_slider = widgets.FloatSlider(
    value=20,
    min=-10,
    max=50,
    step=1,
    description='Temperature (°C):',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='500px')
)

# Arrange sliders vertically with spacing
slider_layout = Layout(display='flex', flex_flow='column', align_items='stretch', width='500px')
sliders = VBox([time_slider, uv_slider, lux_slider, humidity_slider, temp_slider], layout=slider_layout)

# Display the sliders
display(sliders)

def simulate_exposure_by_material(lab_image, art_type, material_type, dye_type, exposure_years, uv_exposure, lux_hours, humidity, temperature):
    lab_exposed = lab_image.copy()

    # Normalize lux_hours and uv_exposure
    lux_normalized = lux_hours / 100000  # Now ranges from 0.0 to 1.0
    uv_normalized = uv_exposure  # Already between 0.0 and 1.0

    # Combined exposure factor
    exposure_factor = lux_normalized + uv_normalized
    exposure_factor = np.clip(exposure_factor, 0, 2)  # Max exposure factor is 2

    # Refined Material and Dye Interactions with Light (Lux Hours and UV)
    if art_type == 'Chromolithograph Print':
        lab_exposed[:, :, 0] -= ((lux_normalized * 10) + (uv_normalized * 10))
        lab_exposed[:, :, 1] -= ((lux_normalized * 5) + (uv_normalized * 5))
        lab_exposed[:, :, 2] -= ((lux_normalized * 5) + (uv_normalized * 5))
    elif art_type == 'Sanguine Etching':
        lab_exposed[:, :, 1] -= (lux_normalized * 10)
    elif art_type == 'Steel Engraving':
        lab_exposed[:, :, 0] -= (lux_normalized * 5)
    elif art_type == 'None':
        # No artwork-specific fading
        pass

    # Adjustments based on material type
    if 'Acidic' in material_type:
        lab_exposed[:, :, 0] -= uv_normalized * 10
        lab_exposed[:, :, 2] += uv_normalized * 10  # Increase yellowing
    elif 'Alkaline' in material_type:
        lab_exposed[:, :, 0] -= lux_normalized * 5
    elif material_type == 'Textiles':
        if dye_type == 'Natural':
            fading_multiplier = np.log(lux_hours + 1) / np.log(100000 + 1)
            lab_exposed[:, :, 0] -= uv_normalized * 15 * fading_multiplier
            lab_exposed[:, :, 1] -= uv_normalized * 15 * fading_multiplier
            lab_exposed[:, :, 2] -= uv_normalized * 15 * fading_multiplier
        elif dye_type == 'Synthetic':
            lab_exposed[:, :, 0] -= uv_normalized * 10
            lab_exposed[:, :, 1] -= uv_normalized * 10
            lab_exposed[:, :, 2] -= uv_normalized * 10
    elif material_type == 'Paper with Black Text':
        lab_exposed[:, :, 0] -= lux_normalized * 2

    # Ensure values stay within valid LAB ranges
    lab_exposed[:, :, 0] = np.clip(lab_exposed[:, :, 0], 0, 100)
    lab_exposed[:, :, 1] = np.clip(lab_exposed[:, :, 1], -128, 127)
    lab_exposed[:, :, 2] = np.clip(lab_exposed[:, :, 2], -128, 127)

    logging.info(f"Simulated exposure for {art_type} on {material_type} with dye type {dye_type}.")
    return lab_exposed

# Added missing function: lab_to_rgb
def lab_to_rgb(lab_image):
    rgb_image = color.lab2rgb(lab_image)
    rgb_image = np.clip(rgb_image, 0, 1)
    rgb_image = (rgb_image * 255).astype(np.uint8)
    return rgb_image

# Added missing function: display_image
def display_image(image, title='Image', save_fig=False, filename=None):
    plt.figure(figsize=(8, 6))
    plt.imshow(image)
    plt.title(title)
    plt.axis('off')
    if save_fig and filename:
        plt.savefig(filename, bbox_inches='tight')
        logging.info(f"Image saved as {filename}")
    plt.show()

# Added missing function: apply_fading
def apply_fading(lab_image, predicted_fading):
    lab_faded = lab_image.copy()
    lab_faded[:, :, 0] += predicted_fading[0]
    lab_faded[:, :, 1] += predicted_fading[1]
    lab_faded[:, :, 2] += predicted_fading[2]
    lab_faded[:, :, 0] = np.clip(lab_faded[:, :, 0], 0, 100)
    lab_faded[:, :, 1] = np.clip(lab_faded[:, :, 1], -128, 127)
    lab_faded[:, :, 2] = np.clip(lab_faded[:, :, 2], -128, 127)
    logging.info("Applied predicted fading to the image.")
    return lab_faded

# -----------------------------
# Step 4: Visualization and Delta-E
# -----------------------------

def compute_delta_e(lab1, lab2):
    delta_e = deltaE_ciede2000(lab1, lab2)
    logging.info(f"Delta-E between the two images calculated.")
    return delta_e

def display_color_difference(delta_e, title='Color Difference Map (∆E)', save_fig=False, filename=None):
    plt.figure(figsize=(8, 6))
    plt.imshow(delta_e, cmap='hot')
    plt.colorbar(label='∆E')
    plt.title(title)
    plt.axis('off')
    if save_fig and filename:
        plt.savefig(filename, bbox_inches='tight')
        logging.info(f"Color difference map saved as {filename}")
    plt.show()

def plot_histograms(image1, image2, title_suffix='', save_fig=False, filename=None):
    import textwrap
    image1_array = np.array(image1)
    image2_array = np.array(image2)
    fig, axs = plt.subplots(1, 3, figsize=(15, 5))
    colors = ['Red', 'Green', 'Blue']
    for i, color_name in enumerate(colors):
        axs[i].hist(image1_array[..., i].flatten(), bins=256, alpha=0.5, label=f'{color_name} (Image 1)', color=color_name.lower())
        axs[i].hist(image2_array[..., i].flatten(), bins=256, alpha=0.5, label=f'{color_name} (Image 2)', color=f'dark{color_name.lower()}')
        # Wrap the title to a maximum width
        wrapped_title = textwrap.fill(f'{color_name} Channel {title_suffix}', width=25)
        axs[i].set_title(wrapped_title, fontsize=10)
        axs[i].legend()
    plt.tight_layout()
    plt.subplots_adjust(wspace=0.4)  # Increase horizontal space between subplots
    if save_fig and filename:
        plt.savefig(filename, bbox_inches='tight')
        logging.info(f"Histogram saved as {filename}")
    plt.show()

def display_average_color(image_lab, title='Average Color', save_fig=False, filename=None):
    average_lab = image_lab.mean(axis=(0,1))
    average_rgb = color.lab2rgb(np.reshape(average_lab, (1,1,3))).reshape(1,1,3)
    average_rgb = np.clip(average_rgb, 0, 1)
    plt.figure(figsize=(2,2))
    plt.imshow(np.ones((100,100,3)) * average_rgb)
    plt.title(title)
    plt.axis('off')
    if save_fig and filename:
        plt.savefig(filename, bbox_inches='tight')
        logging.info(f"Average color saved as {filename}")
    plt.show()
    logging.info(f"{title}: L={average_lab[0]:.2f}, A={average_lab[1]:.2f}, B={average_lab[2]:.2f}")
    return average_lab

# New function to compute Delta-E between average colors
def compute_average_delta_e(avg_lab1, avg_lab2):
    lab1 = np.array([avg_lab1])
    lab2 = np.array([avg_lab2])
    delta_e = deltaE_ciede2000(lab1, lab2)[0]
    logging.info(f"Delta-E between average colors: {delta_e:.2f}")
    return delta_e

# -----------------------------
# Step 5: Main Execution Flow
# -----------------------------

def main():
    # Upload and load dataset
    csv_filename = upload_file("Please upload your LAB color dataset CSV file.", file_types=('.csv',))
    dataset = load_and_clean_dataset(csv_filename)

    # Upload and process image
    original_image, original_lab = upload_and_process_image()

    # Display average color before fading
    avg_lab_before = display_average_color(original_lab, title='Average Color - Original Image', save_fig=True, filename='average_color_before.png')

    # Create synthetic data and train model
    synthetic_data = create_synthetic_data(art_types, material_types, dye_types, valid_combinations, num_samples_per_combination=500)
    X, Y, encoder, poly = prepare_features(synthetic_data)
    model, scaler, mse = train_ml_model(X, Y)
    print(f"Cross-validated Mean Squared Error for Fading Prediction: {mse:.4f}")

    # Get environmental parameters
    art_type = art_type_dropdown.value
    material_type = material_type_dropdown.value
    dye_type = dye_type_dropdown.value if dye_type_dropdown.layout.visibility == 'visible' else 'None'
    exposure_years = time_slider.value
    uv_exposure = uv_slider.value
    lux_hours = lux_slider.value
    humidity = humidity_slider.value
    temperature = temp_slider.value

    # Simulate exposure by material
    lab_exposed = simulate_exposure_by_material(original_lab, art_type, material_type, dye_type, exposure_years, uv_exposure, lux_hours, humidity, temperature)
    exposed_image = lab_to_rgb(lab_exposed)
    display_image(exposed_image, title=f'Simulated Exposure: {art_type} on {material_type}', save_fig=True, filename='exposed_image.png')

    # Display average color after simulated exposure
    avg_lab_exposed = display_average_color(lab_exposed, title='Average Color - Simulated Exposure', save_fig=True, filename='average_color_exposed.png')

    # Compute Delta-E between Original and Simulated Exposure
    delta_e_simulation = compute_delta_e(original_lab, lab_exposed)
    display_color_difference(delta_e_simulation, title='Color Difference Map (∆E) - Original vs Simulated Exposure', save_fig=True, filename='delta_e_simulation.png')
    delta_e_avg_simulation = compute_average_delta_e(avg_lab_before, avg_lab_exposed)
    print(f"Delta-E between average colors (Original vs Simulated Exposure): {delta_e_avg_simulation:.2f}")

    # Plot histograms between Original and Simulated Exposure
    plot_histograms(original_image, exposed_image, title_suffix='Original vs Simulated Exposure', save_fig=True, filename='histograms_simulation.png')

    # Prepare features for prediction
    # One-hot encode the selected art type, material type, and dye type
    categorical_input = pd.DataFrame({'art_type': [art_type], 'material_type': [material_type], 'dye_type': [dye_type]})
    categorical_input = categorical_input.fillna('None')
    categorical_encoded = encoder.transform(categorical_input)

    # Create polynomial features for the numeric input
    X_input_numeric = np.array([[lux_hours, uv_exposure, temperature, humidity]])
    X_input_numeric_poly = poly.transform(X_input_numeric)

    # Combine numeric and categorical features
    X_input = np.hstack((X_input_numeric_poly, categorical_encoded))
    X_input_scaled = scaler.transform(X_input)

    # Predict fading and apply it
    predicted_fading = model.predict(X_input_scaled)[0]
    lab_faded = apply_fading(lab_exposed, predicted_fading)
    faded_image = lab_to_rgb(lab_faded)
    display_image(faded_image, title=f'Faded Image After ML Prediction', save_fig=True, filename='faded_image.png')

    # Display average color after fading
    avg_lab_after = display_average_color(lab_faded, title='Average Color - After ML Prediction', save_fig=True, filename='average_color_after.png')

    # Compute Delta-E between Simulated Exposure and ML Prediction
    delta_e_ml = compute_delta_e(lab_exposed, lab_faded)
    display_color_difference(delta_e_ml, title='Color Difference Map (∆E) - Simulated Exposure vs ML Prediction', save_fig=True, filename='delta_e_ml.png')
    delta_e_avg_ml = compute_average_delta_e(avg_lab_exposed, avg_lab_after)
    print(f"Delta-E between average colors (Simulated Exposure vs ML Prediction): {delta_e_avg_ml:.2f}")

    # Compute Delta-E between Original and Final Faded Image
    delta_e_total = compute_delta_e(original_lab, lab_faded)
    display_color_difference(delta_e_total, title='Color Difference Map (∆E) - Original vs Final Faded', save_fig=True, filename='delta_e_total.png')
    delta_e_avg_total = compute_average_delta_e(avg_lab_before, avg_lab_after)
    print(f"Delta-E between average colors (Original vs Final Faded): {delta_e_avg_total:.2f}")

    # Plot histograms between Original and Final Faded Image
    plot_histograms(original_image, faded_image, title_suffix='Original vs Final Faded', save_fig=True, filename='histograms_final.png')

    # Provide download links if running in Google Colab
    try:
        from google.colab import files
        print("Preparing files for download...")
        files_to_download = [
            'average_color_before.png',
            'exposed_image.png',
            'average_color_exposed.png',
            'delta_e_simulation.png',
            'histograms_simulation.png',
            'faded_image.png',
            'average_color_after.png',
            'delta_e_ml.png',
            'delta_e_total.png',
            'histograms_final.png'
        ]
        for file in files_to_download:
            files.download(file)
    except ImportError:
        print("Files saved locally. Download functionality is not available in this environment.")

# Create a 'Run Simulation' button and output area
run_button = widgets.Button(description='Run Simulation', layout=widgets.Layout(width='200px'))
output = widgets.Output()

def on_button_clicked(b):
    with output:
        output.clear_output()
        main()

run_button.on_click(on_button_clicked)

# Arrange button and output
interface_layout = VBox([run_button, output], layout=Layout(align_items='center'))

# Display the button and output
display(interface_layout)